In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [2]:
base_dir = "data/"

In [3]:
# kurupical
df1 = pd.read_csv(f"{base_dir}/model225/transformers1.csv")
df2 = pd.read_csv(f"{base_dir}/model226/transformers1.csv")
df3 = pd.read_csv(f"{base_dir}/model228/transformers1.csv")
df4 = pd.read_csv(f"{base_dir}/model248/transformers1.csv")


In [4]:
# takoi-san
df_val_row = pd.read_feather("../input/riiid-test-answer-prediction/train_transformer_last2500k_only_row_id.feather")

def extract_val_row(path):
    df = pd.read_csv(path)
    return pd.merge(df, df_val_row, how="inner")

df5 = extract_val_row(f"{base_dir}/ex65_66_67/ex65_66_67_predict.csv")
df6 = extract_val_row(f"{base_dir}/lgb014.csv/lgb014.csv") 

In [5]:
df5.columns, df6.columns

(Index(['row_id', 'ex65_transformer_60m', 'ex66_transformer_60m',
        'ex67_lstm_75m'],
       dtype='object'),
 Index(['row_id', 'ex014_lgb'], dtype='object'))

In [22]:
def sigmoid(x):
    return 1 / (1 + np.e**-x)

In [24]:
# merge
df = pd.DataFrame()
df["target"] = df1["target"]
df["model225"] = df1["predict"]
df["model226"] = df2["predict"]
df["model228"] = df3["predict"]
df["model248"] = df4["predict"]
df["ex65"] = sigmoid(df5["ex65_transformer_60m"])
df["ex66"] = sigmoid(df5["ex66_transformer_60m"])
df["ex67"] = sigmoid(df5["ex67_lstm_75m"])
df["lgb014"] = df6["ex014_lgb"]

# df["ex55"] = df4["ex55"]
# df["ex56"] = df4["ex56"]
# df["ex57"] = df4["ex57"]

In [25]:
from scipy.optimize import minimize

In [26]:
X = df.drop("target", axis=1).values
y = df["target"].values

In [34]:
(X + initial_weight).sum(axis=1)

array([13.80840603, 14.79978137, 11.25808544, ..., 15.31706201,
       10.75345161, 15.83539684])

In [35]:
def auc(weights):
    ''' scipy minimize will pass the weights as a numpy array '''
    weights = weights / weights.sum()
    auc = roc_auc_score(y, (X*weights).sum(axis=1))
    print(auc)
    return -auc

In [36]:
initial_weight = np.ones(len(df.columns)-1)
coef_ = minimize(auc, initial_weight, method='nelder-mead')

0.8137247651631699
0.8137221649614081
0.8137274258619105
0.8137192161545042
0.8137241182884531
0.8137247747458048
0.8137256365676996
0.8137283357549536
0.8137248154428381
0.8137312613538272
0.8137367095281774
0.8137318532585934
0.8137316587297193
0.8137332259071368
0.8137353073915057
0.8137373520153671
0.8137423788586275
0.8137413283942352
0.8137417947414882
0.8137449840124589
0.8137518249720792
0.8137472996235099
0.8137504378468131
0.8137536655337338
0.8137624826006824
0.8137570150839926
0.8137607697297193
0.8137606027106449
0.8137655774806184
0.8137743258164665
0.8137716230454457
0.8137727534408162
0.8137754652979804
0.8137821822215041
0.8137802004493108
0.8137834673090698
0.8137908071531542
0.8137887747846493
0.8137902013785658
0.8137897817411024
0.813788440185431
0.8137924542372467
0.8137869068117988
0.8137881402482688
0.8137905275825631
0.8137928901367194
0.8137844928023656
0.8137893556506367
0.8137956120320401
0.8137958949829407
0.8137898381078261
0.8137871932728599
0.81379296941

In [37]:
coef_

 final_simplex: (array([[0.80837991, 1.65691584, 0.07916617, 1.10564806, 0.73283159,
        1.07527989, 1.40319146, 1.0209718 ],
       [0.8083543 , 1.65699185, 0.07918547, 1.10563878, 0.73288434,
        1.07527568, 1.40323429, 1.0210174 ],
       [0.8083651 , 1.6569887 , 0.07916381, 1.10564072, 0.73286278,
        1.07523124, 1.40324285, 1.02099615],
       [0.8083313 , 1.65692979, 0.0791893 , 1.10568527, 0.73288807,
        1.07529444, 1.40321802, 1.021003  ],
       [0.80834468, 1.6569108 , 0.07918949, 1.10564702, 0.73287373,
        1.07522604, 1.40319242, 1.02098024],
       [0.80837468, 1.65700272, 0.07914342, 1.105631  , 0.73292493,
        1.0752986 , 1.40323382, 1.02100918],
       [0.80834688, 1.65695433, 0.07918487, 1.10562271, 0.73282406,
        1.07528099, 1.40323625, 1.02096335],
       [0.80834324, 1.6569819 , 0.07918231, 1.10567793, 0.73287436,
        1.0753214 , 1.40319558, 1.02103482],
       [0.80835573, 1.65698915, 0.07919737, 1.10562799, 0.73279379,
        1.0

In [41]:
weight_norm = coef_.x / coef_.x.sum()

In [43]:
for i in range(len(weight_norm)):
    print("{}: {}".format(df.columns[i+1], round(weight_norm[i], 4)))

model225: 0.1026
model226: 0.2102
model228: 0.01
model248: 0.1403
ex65: 0.093
ex66: 0.1364
ex67: 0.178
lgb014: 0.1295


In [70]:
weight = coef_.x.copy()

In [71]:
weight[7] = weight[7] + 0.7

In [75]:
weight

array([0.80837991, 1.65691584, 0.07916617, 1.10564806, 0.73283159,
       1.07527989, 1.40319146, 1.7209718 ])

In [76]:
weight_norm = weight / weight.sum()

In [77]:
weight_norm

array([0.09419059, 0.19306007, 0.00922426, 0.1288276 , 0.08538787,
       0.12528917, 0.16349669, 0.20052373])

In [73]:
for i in range(len(weight_norm)):
    print("{}: {}".format(df.columns[i+1], round(weight_norm[i], 4)))

model225: 0.0942
model226: 0.1931
model228: 0.0092
model248: 0.1288
ex65: 0.0854
ex66: 0.1253
ex67: 0.1635
lgb014: 0.2005


In [40]:
roc_auc_score(y[-500000:], (X[-500000:]*coef_.x).sum(axis=1))

0.8114712024913775